In [1]:
import yfinance as yf
import pandas as pd
import functions as f

%load_ext autoreload
%autoreload 2

stock = yf.Ticker("SPY")
dividends = stock.dividends

In [2]:
spot_prices = stock.history(period="1y")["Close"].to_frame()
spot_price = stock.history(period="1d")["Close"].iloc[-1]

In [3]:
daily_returns = spot_prices.pct_change().dropna()

In [4]:
historical_volatility = daily_returns.std() * (252**0.5)

In [5]:
expiration_dates = stock.options

In [6]:
calls_all = [stock.option_chain(date).calls for date in expiration_dates]
puts_all = [stock.option_chain(date).puts for date in expiration_dates]

In [7]:
calls_dict = {date: stock.option_chain(date).calls for date in expiration_dates}
puts_dict = {date: stock.option_chain(date).puts for date in expiration_dates}

In [8]:
from datetime import datetime

expiration_df = pd.DataFrame({'expiration': expiration_dates})
expiration_df['time_to_expiry'] = expiration_df['expiration'].apply(
    lambda x: (datetime.strptime(x, '%Y-%m-%d') - datetime.now()).days / 365
)

In [9]:
# Add expiration column to each DataFrame in calls_dict and puts_dict
for date, df in calls_dict.items():
    df['expiration'] = date

for date, df in puts_dict.items():
    df['expiration'] = date

# Concatenate all DataFrames from calls_dict and puts_dict
calls_all = pd.concat(calls_dict.values())
puts_all = pd.concat(puts_dict.values())

# Combine calls and puts into a single DataFrame
options_data = pd.concat([calls_all, puts_all])

# Select desired columns
calls_data = calls_all[['contractSymbol', 'strike', 'lastPrice', 'impliedVolatility', 'expiration']]


In [10]:
calls_data.iloc[0]

contractSymbol       SPY241107C00400000
strike                            400.0
lastPrice                         170.8
impliedVolatility              2.726566
expiration                   2024-11-07
Name: 0, dtype: object

In [11]:
bs = f.BlackScholes(
    type=f.OptionType.PUT,
    spot=594.7899169921875,
    exercise=105,
    years=0.8931506849315068
,
    volatility=0.11209501132709629
,
    risk_free_rate=0.0442,
    dividend_yield=0
)
bs.price()

0.0

In [12]:
f.BlackScholes.find_implied_volatility(
    type=f.OptionType.CALL,
    spot=spot_price,
    exercise=calls_data.iloc[3237]["strike"],
    years=f.calculate_time_to_expiration(calls_data.iloc[3237]["expiration"]),
    target_price=calls_data.iloc[3237]["lastPrice"],
    risk_free_rate=0.0442,
    dividend_yield=0
)


0.11345790253283242

In [13]:
f.calculate_time_to_expiration(calls_data.iloc[3237]["expiration"])

0.8931506849315068

In [14]:
calls_data.iloc[3237]["strike"]

750.0

In [15]:
calls_data.iloc[3237]["lastPrice"]

1.06

In [16]:
calls_data.iloc[3237]

contractSymbol       SPY250930C00750000
strike                            750.0
lastPrice                          1.06
impliedVolatility              0.130441
expiration                   2025-09-30
Name: 133, dtype: object

In [17]:
type(calls_data)

pandas.core.frame.DataFrame

In [47]:
imp_vol_data = pd.DataFrame(columns=["ContractSymbol","StrikePrice","TimeToExpiry", "ImpliedVolatility"])
imp_vol_data.name = "ImpliedVolatility"
df_index = 0

for i in range(len(calls_data)):
    if f.calculate_time_to_expiration(calls_data.iloc[i]["expiration"]) > 0:
        imp = f.BlackScholes.find_implied_volatility(
        type=f.OptionType.CALL,
        spot=spot_price,
        exercise=calls_data.iloc[i]["strike"],
        years=f.calculate_time_to_expiration(calls_data.iloc[i]["expiration"]),
        target_price=calls_data.iloc[i]["lastPrice"],
        risk_free_rate=0.0442,
        dividend_yield=0
        )
        imp_vol_data.loc[df_index] = [calls_data.iloc[i]['contractSymbol'],calls_data.iloc[i]["strike"],\
                                calls_data.iloc[i]["expiration"],imp]
        df_index += 1           

Newton-Raphson failed: Derivative was zero. Failed to converge after 2 iterations, value is -69968927192105.54., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 2 iterations, value is -148597892393.05664., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 2 iterations, value is -61524345.066369206., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 2 iterations, value is -2223175.8471810054., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 2 iterations, value is -349051.5593717394., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivativ

/home/kali/Projects/Implied Volatility Surface/functions.py:48: RuntimeWarning: overflow encountered in scalar power
  b = (self.risk_free_rate - self.dividend_yield + ((self.volatility**2) / 2)) * self.years


Newton-Raphson failed: Derivative was zero. Failed to converge after 7 iterations, value is 1.156875147231395e+44., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 7 iterations, value is -538302.7580481068., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 5 iterations, value is -3780417950.0291214., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 5 iterations, value is -7.472627866544406e+30., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 4 iterations, value is 0.007586873664022775., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: De

/home/kali/Projects/Implied Volatility Surface/functions.py:48: RuntimeWarning: overflow encountered in scalar power
  b = (self.risk_free_rate - self.dividend_yield + ((self.volatility**2) / 2)) * self.years
/home/kali/Projects/Implied Volatility Surface/functions.py:48: RuntimeWarning: overflow encountered in scalar power
  b = (self.risk_free_rate - self.dividend_yield + ((self.volatility**2) / 2)) * self.years
/home/kali/Projects/Implied Volatility Surface/functions.py:48: RuntimeWarning: overflow encountered in scalar power
  b = (self.risk_free_rate - self.dividend_yield + ((self.volatility**2) / 2)) * self.years


Newton-Raphson failed: Derivative was zero. Failed to converge after 3 iterations, value is -1077.0951147892126., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 4 iterations, value is 1.756288717865714e+21., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 12 iterations, value is 3960.7993526500272., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 8 iterations, value is 1.653189422907946e+180., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 3 iterations, value is 0.005540626684219896., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: D

...

/tmp/ipykernel_9627/3635303534.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  imp_vol_data.loc[df_index] = [calls_data.iloc[i]['contractSymbol'],calls_data.iloc[i]["strike"],\
/tmp/ipykernel_9627/3635303534.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  imp_vol_data.loc[df_index] = [calls_data.iloc[i]['contractSymbol'],calls_data.iloc[i]["strike"],\
/tmp/ipykernel_9627/3635303534.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

Newton-Raphson failed: Derivative was zero. Failed to converge after 5 iterations, value is 5633783867228658.0., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 7 iterations, value is 4.986297633742525e+17., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 6 iterations, value is 0.0028698559624989184., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 10 iterations, value is -2.3800849833511683e+144., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 6 iterations, value is 0.0013646026408520723., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson fail

/tmp/ipykernel_9627/3635303534.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  imp_vol_data.loc[df_index] = [calls_data.iloc[i]['contractSymbol'],calls_data.iloc[i]["strike"],\
/tmp/ipykernel_9627/3635303534.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  imp_vol_data.loc[df_index] = [calls_data.iloc[i]['contractSymbol'],calls_data.iloc[i]["strike"],\
/tmp/ipykernel_9627/3635303534.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

Newton-Raphson failed: Derivative was zero. Failed to converge after 4 iterations, value is 52.621106308435856., switching to Brent's method.
Brent's method also failed after expanding bounds.
Newton-Raphson failed: Derivative was zero. Failed to converge after 5 iterations, value is 1.9729443403621464e+166., switching to Brent's method.
Brent's method also failed after expanding bounds.


/tmp/ipykernel_9627/3635303534.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  imp_vol_data.loc[df_index] = [calls_data.iloc[i]['contractSymbol'],calls_data.iloc[i]["strike"],\
/home/kali/Projects/Implied Volatility Surface/functions.py:48: RuntimeWarning: overflow encountered in scalar power
  b = (self.risk_free_rate - self.dividend_yield + ((self.volatility**2) / 2)) * self.years
/tmp/ipykernel_9627/3635303534.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  imp_vol_data.loc[df_index

In [51]:
imp_vol_data = imp_vol_data.dropna()

In [53]:
imp_vol_data

,ContractSymbol,StrikePrice,TimeToExpiry,ImpliedVolatility
43,SPY241111C00590000,590.0,2024-11-11,0.104890
44,SPY241111C00591000,591.0,2024-11-11,0.098912
45,SPY241111C00592000,592.0,2024-11-11,0.089438
46,SPY241111C00593000,593.0,2024-11-11,0.098301
47,SPY241111C00594000,594.0,2024-11-11,0.104978
...,...,...,...,...
3629,SPY270115C00880000,880.0,2027-01-15,0.106221
3630,SPY270115C00885000,885.0,2027-01-15,0.113112
3631,SPY270115C00890000,890.0,2027-01-15,0.108324
3632,SPY270115C00895000,895.0,2027-01-15,0.108059
